In [1]:
import hash_blocking_spark as hash
import ngram_blocking_spark as ngram
import matchers_spark as match
import similarity_spark as sim
import blocking_structured_and_sorted_spark as ss
import sys

In [2]:
import pandas as pd
dblp_csv = '../CSV-files/dblp_stem.csv'
dblp = pd.read_csv(dblp_csv)

acm_csv = '../CSV-files/acm_stem.csv'
acm = pd.read_csv(acm_csv)


In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkk").getOrCreate()

acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)

selected_columns = ['author_names', 'paper_title']
c = ngram.n_gram_blocking(acm_df, selected_columns,2)
d = ngram.n_gram_blocking(dblp_df, selected_columns,2)
e = match.apply_ngram_blocks_spark(c,d, 0.7, sim.jaccard_similarity_ngrams)


spark.stop

Processing time: 1.8495590686798096 seconds. Number of index combinations: 0


<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x107669690>>

In [29]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkk").getOrCreate()

acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)

selected_columns = ['author_names', 'paper_title']
c = ngram.initial_ngram_parallel_df(spark,acm_df, selected_columns,2)
d = ngram.initial_ngram_parallel_df(spark, dblp_df, selected_columns,2)
e = match.apply_similarity_blocks_spark(c,d, 0.7, sim.jaccard_similarity)
#e = apply_similarity_blocks_spark(c, d, 0.1, sim.jaccard_similarity_ngrams)

spark.stop
#db = ngram.initial_ngram_parallel(dblp_csv, selected_columns, 2)

#simo = match.apply_similarity_blocks_spark(ap, db, 0.7, hash_indices, sim.jaccard_similarity_ngrams)


Processing time: 1.8713901042938232 seconds. Number of index combinations: 782


<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x11b67c460>>

In [63]:
from itertools import product

# Your data
data = [
    {'index1': '[53908b4920f70186a0dbacb7]', 'index2': '[53e9a297b7602d9702b9cf03]', 'similarity': '1.0'},
    {'index1': '[5390958a20f70186a0df0138]', 'index2': '[53e9af2db7602d9703969dc1]', 'similarity': '1.0'},
    {'index1': '[53908a9620f70186a0da4d9c]', 'index2': '[53e9b98ab7602d970457bd03]', 'similarity': '1.0'},
    {'index1': '[539087f320f70186a0d6f876, 53908b4920f70186a0dbb083]', 'index2': '[53e9ad04b7602d97036e13e5]', 'similarity': '1.0'},
    # Add more rows here
]

# Initialize a list to store processed rows
processed_data = []

# Iterate through each row
for row in data:
    # Split the index values and remove brackets
    index1_values = row['index1'][1:-1].split(', ')
    index2_values = row['index2'][1:-1].split(', ')

    # Check if either index has multiple values
    if len(index1_values) > 1 or len(index2_values) > 1:
        # Generate combinations of indices
        index_combinations = list(product(index1_values, index2_values))

        # Create a new row for each combination
        for combination in index_combinations:
            processed_row = {
                'index1': combination[0],
                'index2': combination[1],
                'similarity': row['similarity']
            }
            processed_data.append(processed_row)
    else:
        # If no index has multiple values, simply append the row
        processed_data.append(row)

# Print the processed data
for row in processed_data:
    print(row)


{'index1': '[53908b4920f70186a0dbacb7]', 'index2': '[53e9a297b7602d9702b9cf03]', 'similarity': '1.0'}
{'index1': '[5390958a20f70186a0df0138]', 'index2': '[53e9af2db7602d9703969dc1]', 'similarity': '1.0'}
{'index1': '[53908a9620f70186a0da4d9c]', 'index2': '[53e9b98ab7602d970457bd03]', 'similarity': '1.0'}
{'index1': '539087f320f70186a0d6f876', 'index2': '53e9ad04b7602d97036e13e5', 'similarity': '1.0'}
{'index1': '53908b4920f70186a0dbb083', 'index2': '53e9ad04b7602d97036e13e5', 'similarity': '1.0'}


In [56]:
from itertools import product

# Define your pairs
pairs = [
    {'index1': [1], 'index2': [3]},
    {'index1': [2], 'index2': [4]}

]

combined_pairs = []

# Iterate through each pair
for pair in pairs:
    # Generate combinations of index1 and index2 values
    combinations = list(product(pair['index1'], pair['index2']))

    # Append combinations to the combined_pairs list
    combined_pairs.extend(combinations)

print(combined_pairs)



[(1, 3), (2, 4)]


In [26]:
print(e)

[Row(index1='[53908b4920f70186a0dbacb7]', index2='[53e9a297b7602d9702b9cf03]', similarity='1.0'), Row(index1='[5390958a20f70186a0df0138]', index2='[53e9af2db7602d9703969dc1]', similarity='1.0'), Row(index1='[53908a9620f70186a0da4d9c]', index2='[53e9b98ab7602d970457bd03]', similarity='1.0'), Row(index1='[539087f320f70186a0d6f876, 53908b4920f70186a0dbb083]', index2='[53e9ad04b7602d97036e13e5]', similarity='1.0'), Row(index1='[53909f6920f70186a0e3a999]', index2='[53e9aed1b7602d97038fa413]', similarity='1.0'), Row(index1='[539087dd20f70186a0d63498]', index2='[53e9a839b7602d970318585a]', similarity='1.0'), Row(index1='[53908a9620f70186a0da4d8f]', index2='[53e99ac4b7602d9702343ad0]', similarity='1.0'), Row(index1='[5390880d20f70186a0d7b5ec]', index2='[53e9b30ab7602d9703dce3a4]', similarity='1.0'), Row(index1='[5390979920f70186a0dffb8c]', index2='[53e99a49b7602d97022acf07]', similarity='1.0'), Row(index1='[53908a7420f70186a0da453c]', index2='[53e9a4b8b7602d9702dd5e3b]', similarity='1.0'), Row

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import time
from pyspark.sql.types import StringType
from pyspark.sql.types import StringType, ArrayType, StructField, StructType
def apply_similarity_blocks_spark(blocks1, blocks2, threshold, field_name, similarity_function):
    start_time = time.time()



    df1 = spark.read.csv(blocks1, header=True, inferSchema=True)
    df2 = spark.read.csv(blocks2, header=True, inferSchema=True)
    data_tuples1 = [(key, value[field_name], value['index']) for key, value in blocks1.items()]
    data_tuples2 = [(key, value[field_name], value['index']) for key, value in blocks2.items()]
    
    schema = StructType([       
        StructField("blocking_key", StringType(), nullable=False),
        StructField(field_name, StringType(), nullable=False),
        StructField("index", ArrayType(StringType()), nullable=False)
    ])
    
    blocks1_df = spark.createDataFrame(data_tuples1, schema=schema)
    blocks2_df = spark.createDataFrame(data_tuples2, schema=schema)
  
   
    similarity_udf = F.udf(lambda set1, set2: str(similarity_function(set(set1), set(set2))), StringType())

    joined_blocks = df1.alias("block1").crossJoin(df2.alias("block2"))
    similar_pairs_df = joined_blocks.where(
        (F.col("block1.value") == F.col("block2.value"))
    ).select(
        F.col("block1.index").alias("index1"),
        F.col("block2.index").alias("index2"),
        similarity_udf(F.col("block1.value"), F.col("block2.value")).alias("similarity")
    )

    similar_pairs_df = similar_pairs_df.filter(F.col("similarity") >= threshold)
    similar_pairs = similar_pairs_df.collect()
    
    index_combinations = [(i, j) for pair in similar_pairs for i in pair['index1'] for j in pair['index2']]
    index_combinations = list(set(index_combinations))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Processing time: {elapsed_time} seconds. Number of index combinations: {len(similar_pairs)}")

    return similar_pairs

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkk").getOrCreate()

acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)

selected_columns = ['author_names', 'paper_title']
c = hash.initial_hash_parallel(spark, acm_df, selected_columns)
d = hash.initial_hash_parallel(spark, dblp_df, selected_columns)
print(c.count(), d.coun
e = match.apply_similarity_blocks_spark(c, d, 0.7, sim.jaccard_similarity)

In [54]:
ap.show()

Py4JJavaError: An error occurred while calling o2369.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2390)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2419)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2438)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:530)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:483)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:61)
	at org.apache.spark.sql.Dataset.collectFromPlan(Dataset.scala:4344)
	at org.apache.spark.sql.Dataset.$anonfun$head$1(Dataset.scala:3326)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$2(Dataset.scala:4334)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:546)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:4332)
	at org.apache.spark.sql.execution.SQLExecution$.

In [4]:
+

Py4JJavaError: An error occurred while calling o102.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2390)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2419)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2438)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:530)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:483)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:61)
	at org.apache.spark.sql.Dataset.collectFromPlan(Dataset.scala:4344)
	at org.apache.spark.sql.Dataset.$anonfun$head$1(Dataset.scala:3326)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$2(Dataset.scala:4334)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:546)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:4332)
	at org.apache.spark.sql.execution.SQLExecution$.$

In [ ]:
year_block = [1995,1996,1997, 1998, 1999,2000,2001, 2002, 2003, 2004,2005]
labels = ["1995", "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2003", "2004"]
selected_columns = ['author_names', 'paper_title']
ap = ss.block_by_year_and_publisher(acm_csv, year_block, labels)
db = ss.block_by_year_and_publisher(dblp_csv, year_block, labels)
